In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image, ImageOps
import requests
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import torch
import requests
from tqdm import tqdm
from io import BytesIO
from diffusersgrad import StableDiffusionImg2ImgPipeline
import torchvision.transforms as T
from torchmetrics.multimodal import *
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure
from torchmetrics.image import StructuralSimilarityIndexMeasure
import torchvision.transforms as transforms


from utils import preprocess, recover_image
to_pil = T.ToPILImage()

with open("dataset/prompts.json", "r") as f:
    prompts_dict = json.load(f)
image_names = list(prompts_dict.keys())

image_idxs = [0,6,7,10,13,17,20,21,25,27,32,34,35,36,38,39,40,44,46,47,48,49,51,54,55,57,59,67,68,70,72,77,78,79,
80,81,82,84,87,89,92,94,96,97,98,103,104,107,108,111,112,114,115,116,119,120,123,124,128,129,132,134,
139,140,141,143,144,151,154,155,158,159,165,172,173,175,176,177,178,180,182,184,185,186,187,189,190,192,195,
197,198,200,201,204,206,208,209,211,212,213,214,215,218,219,222,224,227,230,231,237,239,240,241,244,249,252,
253,254,257,263,264,265,266,268,269,271,275,278,281,282,286,289,294,296,298,303,304,305,306,309,314,318,319,
320,322,323,331,332,334,335,341,343,345,349,350,353,355,360,361,370,373,375,376,377,383,393,394,395,397,398]

# A fixed random selected seed in all the experiments
SEED = 9209
torch.manual_seed(SEED)

torch.cuda.empty_cache()

transform = transforms.Compose([
    transforms.PILToTensor()
])

metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
fid = FrechetInceptionDistance()
inception = InceptionScore()
psnr = PeakSignalNoiseRatio(data_range=1.0)
ms_ssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0)
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

YES
000000276284.jpg
dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])


In [2]:
# Generated images by adversarial samples
folder = "data/variation/resnet_generate/"
# Generated images by original samples
folder_ori_generate = "data/variation/ori_image_generate/"
# Original images
folder_ori = "dataset/images/"

In [5]:
#PSNR
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)/255.0
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)/255.0
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = psnr(image_advs, image_oris)

print("PSNR: ", score)

torch.Size([500, 3, 512, 512])
PSNR:  tensor(13.7023)


In [ ]:
#FID
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori + image_names[image_idx]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.uint8)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.uint8)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
fid.update(image_advs, real=False)
fid.update(image_oris, real=True)
torch.manual_seed(SEED)
score = fid.compute()
score = score.detach().cpu().numpy()
print("FID: ", score)

In [ ]:
#IS
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori + image_names[image_idx]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.uint8)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.uint8)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
inception.update(image_advs)
torch.manual_seed(SEED)
score = inception.compute()
IS = score[0].detach().cpu().numpy()
print("IS: ", IS)

In [ ]:
#SSIM
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = ssim(image_advs, image_oris)

print("SSIM: ", score)

In [ ]:
#MSSSIM
msssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0)
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = msssim(image_advs, image_oris)

print("MSSSIM: ", score)

In [ ]:
#CLIP Score
total = 0
scores = 0.0
num = 100
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    if total % 20 == 0:
        print(total)
    prompts = prompts_dict[image_names[image_idx]]
    for j in range(5):
        image = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        prompt = prompts[j]
        torch.manual_seed(SEED)
        score = metric(transform(image), prompt)
        score = score.detach().cpu().numpy()
        scores += score
    total += 1
print(scores/(num*5))